In [1]:
#Get the mongo table where are stored the games

import pymongo
client = pymongo.MongoClient()
db = client.game

#Create Mongo table for new games
mongoTable = db["TTGames_EUW"]

In [2]:
#Need to classify teams as following jungler meta or hypercarry meta
#Intuition : hypercarry meta team needs a support, which usually has a specific support item
#            A player having one of those specific items are support
#            A team having one (or more) support player follows hypercarry meta, a team which doesn't follows the jungler meta

#List of support items (as of LoL season 8, up to patch 8.18)
supportItems = [3303, 3309, 3098, 3096, 3097, 3312, 3092, 3311, 3307, 3301, 3306, 3401, 3310, 3304, 3302, 4301, 4302, 3308, 3069, 3305]

#Detect if a player has a support item, and tag it has support
def isSupport(participant):
    for i in range(0,7):
        if participant['stats']['item'+str(i)] in supportItems:
            return True
    return False

#Check if the specified team has a support
def hasSupport(game, teamId):
    for p in game['participants']:
        if isSupport(p) and p["teamId"]==teamId:
            return True
    return False

In [3]:
#Get the ratio of counterjungling, the number of monsters killed in the enemy jungle compared to those in the allied jungle
def getCounterjunglingRatio(game, teamId):
    alliedJungle = 0
    enemyJungle = 0
    for p in game["participants"]:
        if p["teamId"] == teamId:
            alliedJungle += p["stats"]["neutralMinionsKilledTeamJungle"]
            enemyJungle += p["stats"]["neutralMinionsKilledEnemyJungle"]
    return enemyJungle / (enemyJungle+alliedJungle) if (enemyJungle+alliedJungle) > 0 else 1

In [4]:
#Analysis for games since patch 8.15
startTimestamp = 1533097800 * 1000

In [5]:
counterjunglingRatio = []
#Only taking games during less than 20 minutes
for game in mongoTable.find( { "gameCreation": { "$gt": startTimestamp }, "gameDuration": {"$lt" : 1200} }):
    #Only counting games with hypercarry VS jungle, getting the CJ ratio of the hypercarry team
    if hasSupport(game, 100) and not hasSupport(game, 200):
        counterjunglingRatio.append(getCounterjunglingRatio(game, 100))
    if hasSupport(game, 200) and not hasSupport(game, 100):
        counterjunglingRatio.append(getCounterjunglingRatio(game, 200))

In [6]:
import pandas as pd
serie = pd.Series(counterjunglingRatio)
print("Mean :   " + str(serie.mean()))
print("Median : " + str(serie.median()))
print("Min :    " + str(serie.min()))
print("Max :    " + str(serie.max()))
print("0.25 :   " + str(serie.quantile(0.25)))
print("0.75 :   " + str(serie.quantile(0.75)))

Mean :   0.181565771846
Median : 0.180327868852
Min :    0.0
Max :    0.708333333333
0.25 :   0.0
0.75 :   0.2970843183609141
